In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib widget
import seaborn as sns
sns.set()  # set plot styles
import sys
import os
from pathlib import Path
from scipy.signal import convolve
from scipy.stats import zscore
from scipy.ndimage import gaussian_filter1d
# from scipy.stats import zscore ### Cave! scipy zscore does not handle NaN!
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from tqdm import tqdm

import rastermap as rm
import pandas as pd
import pickle

from sklearn.decomposition import NMF

### on-the-fly panda function definition of zscore that handles NaNs

In [3]:
def z_score(df): return (df-df.mean())/df.std(ddof=0)
# def m_mean(df): return df.mean()

In [4]:
plt.close('all')

# load panda dataframe, Stim-IDs and one aux from disk

In [8]:
dataframe = pd.read_pickle("./layer1_and_4.pkl")

In [8]:
with open('Chirp_ids.pkl', 'rb') as handle:
    ids = pickle.load(handle)

In [11]:
auxdata = ids[0]['auxdata']

# generate sorted mean-cell responses / zscores or not

In [12]:
# generate sorted mean-cell responses / zscores or not

allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Ipsi_mean_zscored = allcells_Ipsi_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Ipsi_mean_zscored.max().sort_values(ascending=False).index
allcells_Ipsi_mean_sort_zscored = allcells_Ipsi_mean_zscored.reindex(sortidx, axis = 1)

allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Contra_mean_zscored = allcells_Contra_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Contra_mean_zscored.max().sort_values(ascending=False).index
allcells_Contra_mean_sort_zscored = allcells_Contra_mean_zscored.reindex(sortidx, axis = 1)

allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Bino_mean_zscored.max().sort_values(ascending=False).index
allcells_Bino_mean_sort_zscored = allcells_Bino_mean_zscored.reindex(sortidx, axis = 1)


allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Ipsi_mean.max().sort_values(ascending=False).index
allcells_Ipsi_mean_sort = allcells_Ipsi_mean.reindex(sortidx2, axis = 1)

allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Contra_mean.max().sort_values(ascending=False).index
allcells_Contra_mean_sort = allcells_Contra_mean.reindex(sortidx2, axis = 1)

allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Bino_mean.max().sort_values(ascending=False).index
allcells_Bino_mean_sort = allcells_Bino_mean.reindex(sortidx2, axis = 1)


# allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Ipsi_mean_zscored = allcells_Ipsi_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Ipsi_mean_zscored.max().sort_values(ascending=False).index
# allcells_Ipsi_mean_sort_zscored = allcells_Ipsi_mean_zscored.reindex(sortidx, axis = 1)

# allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Contra_mean_zscored = allcells_Contra_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Contra_mean_zscored.max().sort_values(ascending=False).index
# allcells_Contra_mean_sort_zscored = allcells_Contra_mean_zscored.reindex(sortidx, axis = 1)

# allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Bino_mean_zscored.max().sort_values(ascending=False).index
# allcells_Bino_mean_sort_zscored = allcells_Bino_mean_zscored.reindex(sortidx, axis = 1)


# allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# sortidx2 = allcells_Ipsi_mean.max().sort_values(ascending=False).index
# allcells_Ipsi_mean_sort = allcells_Ipsi_mean.reindex(sortidx2, axis = 1)

# allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# sortidx2 = allcells_Contra_mean.max().sort_values(ascending=False).index
# allcells_Contra_mean_sort = allcells_Contra_mean.reindex(sortidx2, axis = 1)

# allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
# sortidx2 = allcells_Bino_mean.max().sort_values(ascending=False).index
# allcells_Bino_mean_sort = allcells_Bino_mean.reindex(sortidx2, axis = 1)

NameError: name 'dataframe' is not defined

### prepare for NMF

In [ ]:
# all cells
sp_ipsi = allcells_Ipsi_mean_zscored.to_numpy()
sp_contra = allcells_Contra_mean_zscored.to_numpy()
sp_bino = allcells_Bino_mean_zscored.to_numpy()

sp_all = np.concatenate((allcells_Contra_mean.to_numpy(), allcells_Ipsi_mean.to_numpy(), allcells_Bino_mean.to_numpy()), axis = 0)

sp_ipsi[np.isnan(sp_ipsi)] = 0
sp_contra[np.isnan(sp_contra)] = 0
sp_bino[np.isnan(sp_bino)] = 0

sp_all[np.isnan(sp_all)] = 0

sp_all = zscore(sp_all) # needs to be done in numpy to be element-wise here - I collapsed the major categories. I dont think I can do that in pandas straight

In [ ]:
%%capture
model_ipsi = rm.mapping.Rastermap(n_components=1).fit(sp_ipsi.T)
model_contra = rm.mapping.Rastermap(n_components=1).fit(sp_contra.T)
model_bino = rm.mapping.Rastermap(n_components=1).fit(sp_bino.T)
model_all = rm.mapping.Rastermap(n_components=1).fit(sp_all.T)

In [ ]:
### sort neurons and smooth across neurons 
smooth = True
timesmooth = True

if smooth:
    isort = np.argsort(model_all.embedding[:,0])   
    Sm_ipsi = gaussian_filter1d(sp_ipsi.T[isort,:], np.minimum(3,int(sp_ipsi.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = gaussian_filter1d(sp_contra.T[isort,:], np.minimum(3,int(sp_contra.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = gaussian_filter1d(sp_bino.T[isort,:], np.minimum(3,int(sp_bino.shape[0]*0.005)), axis=0)
    Sm_all  = gaussian_filter1d(sp_all.T[isort,:], np.minimum(3,int(sp_ipsi.shape[0]*0.005)), axis=0)
#     Sm_all= sp_all.T[isort,:]
else:
    isort = np.argsort(model_bino.embedding[:,0])
    Sm_ipsi = sp_ipsi.T[isort,:]
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = sp_contra.T[isort,:]
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = sp_bino.T[isort,:]
    Sm_all  = sp_all.T[isort,:]

    
if timesmooth:
    sigma = 0.5
# (optional) smooth in time
    Sm_contra = gaussian_filter1d(Sm_contra, sigma, axis=1)
    Sm_ipsi = gaussian_filter1d(Sm_ipsi, sigma, axis=1)
    Sm_bino = gaussian_filter1d(Sm_bino, sigma, axis=1)
    Sm_all = gaussian_filter1d(Sm_all, sigma, axis=1)

### auxdata and timebase 

In [ ]:
slice_range = np.arange(ids[0]['Ipsi'][0][1].astype(int)-prestim_frames[0].astype(int),ids[0]['Ipsi'][1][1].astype(int)+poststim_frames[0].astype(int)+1)
# slice_range = np.arange(ids[category][0][trial].astype(int)-prestim_frames.astype(int),ids[category][1][trial].astype(int)+poststim_frames.astype(int))

aligned_stimdata = auxdata[ids[0]['FrameTimes_level'][slice_range],stimops['Stims_chan']]
# aligned_stimdata_concat = np.tile(auxdata[ids['FrameTimes_level'][slice_range],stimops['Stims_chan']], (0,3))
aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / (ids[0]['framerate_auxderived'] / level), 1 / (ids[0]['framerate_auxderived'] / level) ) - prestim[0]

aligned_stimdata_full = ids[0]['auxdata'][ids[0]['FrameTimes_level'][slice_range[0]]:ids[0]['FrameTimes_level'][slice_range[-1]],stimops['Stims_chan']]
aligned_stimdata_full_concat = np.tile(aligned_stimdata_full, (1,3)).T

aux_time_full = np.arange(0, aligned_stimdata_full.shape[0] * 1 / ids[0]['aux_samplingfreq'] , 1 / ids[0]['aux_samplingfreq']    ) - prestim[0]
aux_time_full_concat = np.arange(0, aligned_stimdata_full_concat.shape[0] * 1 / ids[0]['aux_samplingfreq'] , 1 / ids[0]['aux_samplingfreq']   ) - prestim[0]

# plt.figure()
# plt.plot(aux_time, aligned_stimdata)
# plt.plot(aux_time_full_concat, aligned_stimdata_full_concat)

### seperately sorted

In [ ]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = ids[0]['framerate_auxderived'] / level
vmin_glob = -.5
vmax_glob = 2.5
ncells = cellglob 
with sns.axes_style('white'):
    xl = [-prestim[0],Sm_contra.shape[1]/fs-prestim[0]]
    
    fig8 = plt.figure(figsize=(20,16), constrained_layout=True)
    gs1 = fig8.add_gridspec(nrows=8, ncols=3)
    
    fig8.add_subplot(gs1[1:, :-2])      
    fig8_ax=plt.imshow(Sm_contra, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_contra.shape[1]/fs-prestim[0], 0,Sm_contra.shape[0]], cmap = 'Blues')
    plt.xlabel('time (s)', fontsize=18)    
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0] ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
        
    f8_ax2 = fig8.add_subplot(gs1[1:, 1:-1])
    ax=plt.imshow(Sm_ipsi, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_ipsi.shape[1]/fs-prestim[0], 0,Sm_ipsi.shape[0]], cmap = 'Reds')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0] ],0,ncells-1, 'k')
        
    f8_ax3 = fig8.add_subplot(gs1[1:, 2:])
    ax=plt.imshow(Sm_bino, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_bino.shape[1]/fs-prestim[0], 0,Sm_bino.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,ncells-1, 'k')

#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f8_ax4 = fig8.add_subplot(gs1[:1, :-2])    
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f8_ax4.set_xlim(*xl)
    f8_ax4.axis('off')
    plt.title('Contra')
    
    f8_ax5 = fig8.add_subplot(gs1[:1, 1:-1])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax5.set_xlim(*xl)
    f8_ax5.axis('off')
    plt.title('Ipsi')

    
    f8_ax6 = fig8.add_subplot(gs1[:1, 2:])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax6.set_xlim(*xl)
    f8_ax6.axis('off')
    plt.title('Bino')    
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved Chirp PSTHs - rastermap embedding (sorted on Bino, smoothened over boutons) - exp' + exp[0])
    
    plt.show()

In [ ]:
plt.savefig('separate.eps', format='eps')

In [143]:
ncells

8931

### concatenated figure

In [144]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = ids[0]['framerate_auxderived'] / level
vmin_glob = -.5
vmax_glob = 2.5


with sns.axes_style('white'):
    xl = [-prestim[0],Sm_all.shape[1]/fs - prestim[0]]
    
    fig9 = plt.figure(figsize=(20,20), constrained_layout=True)
    
    gs1 = fig9.add_gridspec(nrows=12, ncols=3)
    
    fig9.add_subplot(gs1[1:, :])      
    fig9_ax=plt.imshow(Sm_all, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_all.shape[1]/fs-prestim[0], 0,Sm_all.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)    
    
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
    
    plt.vlines([aux_time_full[-1],  +aux_time_full[-1] *2 + poststim[0]],0,ncells-1, 'r')
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,ncells-1, 'b')
    
    plt.vlines([aux_time_full[-1] + (prestim_frames[0] / fs),  aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) ],0,ncells-1, 'b')
    plt.vlines([2 * aux_time_full[-1] + (prestim_frames[0] / fs) + prestim[0],  2 * aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) + prestim[0]],0,ncells-1, 'b')
#     plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
    
#     plt.colorbar(label='deconvolved F [zscored]')
#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f9_ax4 = fig9.add_subplot(gs1[:1, :])    
    plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f9_ax4.set_xlim(*xl)
    f9_ax4.axis('off')
#     plt.title('Contra                                    Ipsi                                    Bino')
    plt.text(aux_time_full[-1] / 2 - 2, 5 , 'Contra', fontsize=16)
    plt.text(aux_time_full[-1] * 1.5-2, 5 , 'Ipsi', fontsize=16)
    plt.text(aux_time_full[-1] * 2.5-2, 5 , 'Bino', fontsize=16)
    
   
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved F - Chirp PSTHs - rastermap embedding (smoothened over boutons) - exp' + exp[0])
    
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.savefig('all.eps', format='eps')